## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-25-02-32-42 +0000,bbc,"France will recognise Palestinian state, Macro...",https://www.bbc.com/news/articles/ckg5g4p3245o
1,2025-07-25-02-19-23 +0000,startribune,Dumped tapioca pearls and a staff beatdown: Di...,https://www.startribune.com/dumped-tapioca-pea...
2,2025-07-25-02-18-43 +0000,bbc,Justice department official meets Epstein acco...,https://www.bbc.com/news/articles/cjd2de8zz5go
3,2025-07-25-02-14-01 +0000,bbc,Rescue under way for three workers trapped by ...,https://www.bbc.com/news/articles/cd787n4pqxro
4,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,46
60,new,16
45,he,14
19,epstein,14
281,trade,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
18,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...,137
4,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...,104
19,2025-07-25-01-00-00 +0000,wsj,Buoyed by voters’ improving views of the econo...,https://www.wsj.com/politics/policy/trump-appr...,99
38,2025-07-24-23-54-32 +0000,nyt,Powell Tells Trump He Was Wrong About Renovati...,https://www.nytimes.com/2025/07/24/us/politics...,99
192,2025-07-24-17-07-10 +0000,nypost,Trump threatens to escalate trade war with new...,https://nypost.com/2025/07/24/business/trump-t...,99


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
18,137,2025-07-25-01-00-00 +0000,wsj,Streetwise: The Trump administration has rampe...,https://www.wsj.com/economy/central-banking/tr...
302,73,2025-07-24-07-33-58 +0000,nypost,Thailand and Cambodia exchange fire in new bor...,https://nypost.com/2025/07/24/world-news/thail...
143,66,2025-07-24-19-57-36 +0000,cbc,"Israel, U.S. pull out of Gaza ceasefire talks ...",https://www.cbc.ca/news/world/france-macron-pa...
43,54,2025-07-24-23-50-13 +0000,nypost,Former top Biden White House official blames M...,https://nypost.com/2025/07/24/us-news/former-t...
48,49,2025-07-24-23-28-48 +0000,nypost,Bubba the Love Sponge regrets not making up wi...,https://nypost.com/2025/07/24/us-news/bubba-th...
238,47,2025-07-24-14-19-43 +0000,nypost,Tesla stock plunges as Elon Musk warns company...,https://nypost.com/2025/07/24/business/tesla-s...
4,44,2025-07-25-02-10-00 +0000,wsj,Republican National Committee Chair Michael Wh...,https://www.wsj.com/politics/elections/north-c...
67,43,2025-07-24-22-32-55 +0000,nypost,Zohran Mamdani’s NYC grocery stores scheme dra...,https://nypost.com/2025/07/24/us-news/zohran-m...
90,39,2025-07-24-21-58-32 +0000,latimes,FCC approves Paramount-Skydance merger followi...,https://www.latimes.com/entertainment-arts/bus...
305,39,2025-07-24-06-50-43 +0000,nypost,"Plane crashes in Russia’s Far East, killing al...",https://nypost.com/2025/07/24/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
